In [ ]:
%matplotlib inline
import seaborn as sns
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import pysal as ps
import mplleaflet
import shapely.geometry as geom

In this lab, we're going to review geometry processing techniques with GeoPandas (and Shapely). The objectives are:
<ol type="1">
<li>Given a set of bus location pings, we would like to know its total travel distance.</li>
<li>Providing two shapes of the M1 and M101 bus line, which shape better match our bus location pings.</li>
<li>We also want to determine which neighborhood the bus line in (2) is supposed to pass by.</li>
</ol>

In [ ]:
# For objective (1), we first read in the pings

pings = pd.read_csv('bus_pings.csv')
pings.head()

In [ ]:
# We're going to use the longitude and latitude columns to 
# construct a set of points and visualize them

longlat = pings[['longitude', 'latitude']]
points = map(geom.Point, longlat.as_matrix().tolist())
gpos = gpd.GeoSeries(points)
gpos.plot()

In [ ]:
# In order to compute the travel distance, we need to work
# with LineString instead of just points.

path = geom.LineString(longlat.as_matrix().tolist())
gpath = gpd.GeoSeries(path)
gpath.plot()

In [ ]:
# GeoPandas provides a length property to store the
# total length of a line string. We could use that
# for the first objective

gpath.length

In [ ]:
# However, this doesn't look quite right. The problem
# is that the coordinates were in lat and long, thus,
# the unit of the length is actually in degree! We
# need to project the geometry into a planar projection
# system to get a human readable length. We're going to
# use the NAD83 New York Long Island Projection system,
# EPSG 2263.

# this is the source projection (WGS84 in longlat)
epsg4326 = {'init': 'epsg:4326'}

gpos.crs = epsg4326
projectedPos = gpos.to_crs(epsg=2263)

gpath.crs = epsg4326
projectedPath = gpath.to_crs(epsg=2263)
projectedPath.length

In [ ]:
# For the second objective, we first read the shape of
# the M1 and M101 bus

def readShape(filename):
    data = pd.read_csv(filename)
    ptns = data[['shape_pt_lon', 'shape_pt_lat']]
    line = geom.LineString(ptns.as_matrix().tolist())
    return gpd.GeoSeries(line, crs=epsg4326)
    
m1 = readShape('m1_route.csv')
m101 = readShape('m101_route.csv')
m1.plot()
m101.plot()

In [ ]:
# To find which bus line our location pings belong to,
# we compute the distance between our bus ping and the
# shapes of M1 and M101 buses.

d1 = [m1.to_crs(epsg=2263).distance(p)[0] for p in projectedPos]
d101 = [m101.to_crs(epsg=2263).distance(p)[0] for p in projectedPos]

In [ ]:
# Based on the maximum distance between the pings and the shape,
# we can conclude that the pings are from the M101 bus line

max(d1), max(d101)

In [ ]:
# For the last objective, we can find which neighborhoods
# the M101 bus line is going through by performing an
# intersection test between the neighborhood boundaries and
# the M101 shape.

neighborhoods = gpd.read_file('neighborhood_nta_census.geojson')
intersections = neighborhoods.intersection(m101[0])
intersections

In [ ]:
# We can see that there're a few neighborhoods are
# intersected. Next, we're going to list their names.

names = dict([(neighborhoods.ix[i]['ntaname'],g) 
              for i,g in enumerate(intersections)
              if type(g)!=geom.GeometryCollection])
names.keys()